# qual a quantidade média de auto informação por palavra e de informação por mensagem em cada dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def auto_informacao(i):
    """
    Essa funçãto calcula a autoinformação de um determinado símbolo produzido por uma fonte binária X.
    :param i: Probabilidade de um símbolo ser transmitido pela fonte X. Pode ser um número ou um mapa de :func:`probabilidades`.  
    :return: Informação em bits
    """

    if type(i) != dict:
        return round(-1 * np.log2(i), 4)
    else:
        r = {}
        for key, value in i.items():
            r[key] = round(-1 * np.log2(value), 4)
        return r


## ou informação média
def entropia(c):
    """
    Essa função calcula a entrópia de uma fonte. A entrópia de uma fonte descreve a quantidade média de informação que uma fonte pode transmitir.
    :param c: Mapa de probabilidade de todos os símbolos produzidos por uma fonte binária X. 
    :return: A média ponderada da auto-informacao de todos os símbolos produzidos por uma fonte binária X.
    """
    try:
        c = list(c.values())
    except TypeError:
        raise Exception("c não é um mapa")

    if not c is np.ndarray:
        c = np.array(c)

    return round(sum(np.array(c) * np.log2(1 / np.array(c))), 4)


def probabilidades(c):
    """.value_counts
    Função auxiliar para calcular a probabilidade de ocorrência de cada símbolo de uma coleção.
    :param c: Coleção de símbolos com probabilidade de ocorrência sem dependências.
    :return: Mapa onde as chaves são cada item único da coleção c e os valores são as probabilidades de ocorrência de cada símbolo em relação a todos os outros items.
    """
    try:
        iter(c)
    except TypeError:
        raise Exception("c não é uma lista")

    quantidade_simbolos = {}
    for i in c:
        if quantidade_simbolos.get(i) is None:
            quantidade_simbolos[i] = 1
        else:
            quantidade_simbolos[i] = quantidade_simbolos[i] + 1

    probabilidade_simbolos = {}

    for i in list(set(c)):
        probabilidade_simbolos[i] = quantidade_simbolos[i] / len(c)
    return probabilidade_simbolos

In [3]:
df = pd.read_csv('NoThemeTweets.csv')

In [4]:
print("Analisando cinco tweets:")
print("\n")
df_analise = df[df["tweet_text"].str.len() > 100].sample(5)
for i in range(5):
    mensagem = df_analise.iloc[i].tweet_text
    print(mensagem)
    print("Informação por palavra:")
    probabilidades_por_palavra = probabilidades(mensagem.split())
    informacao_por_palavra = auto_informacao(probabilidades_por_palavra)
    print(informacao_por_palavra)
    max_i = -1
    max_p = None
    for key, value in informacao_por_palavra.items():
        if value > max_i:
            max_i = value
            max_p = key
    print(f"Palavra com mais informação: \"{max_p}\". {max_i} bits/palavra")
    print("Entropia do texto:")
    print(entropia(probabilidades_por_palavra))
    
    print("\n")

Analisando cinco tweets:


Quanto de altura você tem ? — ai tocou em um assunto delicado...... tenho 1,50 :( https://t.co/9bjmFhkLLw
Informação por palavra:
{'Quanto': 4.0875, 'tenho': 4.0875, ':(': 4.0875, 'um': 4.0875, '1,50': 4.0875, 'em': 4.0875, 'tem': 4.0875, 'ai': 4.0875, 'tocou': 4.0875, '—': 4.0875, 'você': 4.0875, 'de': 4.0875, 'altura': 4.0875, 'assunto': 4.0875, 'https://t.co/9bjmFhkLLw': 4.0875, 'delicado......': 4.0875, '?': 4.0875}
Palavra com mais informação: "Quanto". 4.0875 bits/palavra
Entropia do texto:
4.0875


@Sillywalkmary @gaitadaria Infelizmente não aprovaram a outra lei, por isso referia-me somente à Mochi :(
Informação por palavra:
{'@gaitadaria': 3.9069, 'somente': 3.9069, 'a': 3.9069, 'isso': 3.9069, ':(': 3.9069, 'aprovaram': 3.9069, 'outra': 3.9069, 'por': 3.9069, 'à': 3.9069, 'Infelizmente': 3.9069, 'referia-me': 3.9069, 'Mochi': 3.9069, 'lei,': 3.9069, '@Sillywalkmary': 3.9069, 'não': 3.9069}
Palavra com mais informação: "@gaitadaria". 3.9069 bits/pala

# existe uma variação nessas medidas quando os twittes são neutros, positivos ou negativos?

In [5]:
# df = df.sample(20000)
df["entropia"] = df["tweet_text"].apply(lambda x: entropia(probabilidades(x)))
df["entropia"] = df["tweet_text"].apply(lambda x: entropia(probabilidades(x)))

In [6]:
df.groupby("sentiment")["entropia"].mean().sort_values(ascending=False)

sentiment
Positivo    4.184230
Negativo    4.061275
Name: entropia, dtype: float64

In [7]:
df.groupby("sentiment")["entropia"].median().sort_values(ascending=False)

sentiment
Positivo    4.1835
Negativo    4.0585
Name: entropia, dtype: float64

Podemos ver que na media e na mediana a entropia de textos positivos é ligeiramente maior.

# o que é possível afirmar com relação a twiites com temas, usando informação como medida?

Que tweets negativos tendem a ter mais entropia, pois tem palavras mais diversas e buscam passar mais informação.

# se é possível detectar novos usuários usando medidas de informação como parâmetro?

In [8]:
df3 = df[["tweet_date", "entropia"]].sort_values(by="tweet_date").reset_index(drop=True)

In [9]:
df3.sort_values(by="tweet_date", ascending=False, inplace=True)

In [10]:
df3[:10]

,tweet_date,entropia
785813,Wed Sep 26 23:59:52 +0000 2018,4.4088
785812,Wed Sep 26 23:59:42 +0000 2018,4.5796
785811,Wed Sep 26 23:59:41 +0000 2018,4.0653
785810,Wed Sep 26 23:59:41 +0000 2018,4.4498
785809,Wed Sep 26 23:59:41 +0000 2018,4.2459
785808,Wed Sep 26 23:59:40 +0000 2018,4.8008
785807,Wed Sep 26 23:59:37 +0000 2018,3.6645
785806,Wed Sep 26 23:59:30 +0000 2018,4.1621
785805,Wed Sep 26 23:59:29 +0000 2018,4.2201
785804,Wed Sep 26 23:59:24 +0000 2018,3.8316


In [11]:
df3[-10:]

,tweet_date,entropia
9,Fri Aug 17 00:00:22 +0000 2018,4.1072
8,Fri Aug 17 00:00:11 +0000 2018,4.7882
6,Fri Aug 17 00:00:07 +0000 2018,4.6831
7,Fri Aug 17 00:00:07 +0000 2018,4.5188
5,Fri Aug 17 00:00:06 +0000 2018,4.5388
4,Fri Aug 17 00:00:04 +0000 2018,4.5754
2,Fri Aug 17 00:00:04 +0000 2018,4.5146
3,Fri Aug 17 00:00:04 +0000 2018,3.9764
1,Fri Aug 17 00:00:01 +0000 2018,4.7664
0,Fri Aug 17 00:00:00 +0000 2018,5.0296


Não foi possível traçar uma correlação da data dos tweets com a sua entropia